# How to write a Digital Connector importer

Our aim is to write an importer for fetching Body Mass Index for the obese (BMI - 30/39.9 kg/m^2) data from Sports England website, to be used within the Tombolo Digital Connector framework. The data comes as a downloadable .csv file at a local authority level for the whole UK. Our importer will essentially do the following:


* Fetch the dataset from the relevant link. The link is the result of a query that we have already made online using Sport England’s Active People Interactive query builder. Although Tombolo Digital Connector importers allow for a dynamic query building framework we will be using a simple hardcoded link to fetch the data set.

* Import the attributes of interest and assign the correct geometry for every local authority in our dataset.

* Save the data in the Digital Connector Postgres database. The correct formatting and housekeeping will be done automatically by the DC.

Once our importer is ready, we will be able to use it in all our Digital Connector recipes. Note that there is already a generic csv importer that can be used in this case, but for the sake of demonstration we will implement the importer step-by-step as the implementation generalises to other data types (excel files, json files etc.)


Some wordbusting that i will be using throughout this tutorial:
 * **Subject**: You can think of this as a row in our dataset. A DC subject is an object having attributes such as ID, name, and most importantly a spatial extent or Geometry
 * **Attribute**: You can think of this as the columns in our dataset. This is an object that essentially holds the names of our columns along with some other metadata about our columns (such as description). The attributes will appear in our Postgres database as we named them during importing.
 * **Values**: The actual values in the dataset for a particular attribute that we want to import.
 
I will be using IntelliJ IDEA for this tutorial. IntelliJ IDEA is a free Java IDE that has lots of cool functionalities (such as autocomplete, class references, syntax checking, etc.) that will make our lives easier. A few assumptions at this point:

* You have downloaded and installed IntelliJ IDEA 
* You have cloned the Tombolo Digital Connector repo and successfully installed all the dependencies
* You have basic coding experience preferably with Java

I will be using lots of gifs in this tutorial because they’re great.

***Lets get started!***

## Step 1: Download the data from Sport England


* Go to Sport England query builder http://activepeople.sportengland.org/Query and click Physical Activity/Geographies/Local Authorities/select all/OK. 
* Select Local authorities and click OK
* Now click Measures/Body Mass Index (BMI)/Obese/OK
* No click the GO icon which will navigate you to the data
* We need the link to the data. To get that right click on Export table as CSV and click copy link. The link to the query is: 

http://activepeople.sportengland.org/Result/ExportTable?Id=101891&TabDimension=1&RowDimension=2&ColDimension=4&SelectedTabs[0]=39991&ValueMode=0


In [2]:
from IPython.display import HTML
HTML('<img src="gifsImporterTut/output_wKg9Hy.gif">')

## Step 2: Create the importer Java class in IntelliJ


* Open IntelliJ and navigate to the TomboloDigitalConnector folder. Under importer right click and create a new package (this is essentially a folder that will hold our importer classes). Name your package as sportsen (stands for Sports England).

In [6]:
HTML('<img src="gifsImporterTut/output_tAne7U.gif" height="500" width="500">')

* Create a java class by right clicking on the newly created sportsen folder and 
 click new java class. Name this as ObeseImporter. You will be asked if you want to 
 add the file to Git, say no for the time being


In [7]:
HTML('<img src="gifsImporterTut/output_HNkL2I.gif" height="500" width="500">')

## Step 3: Create all the methods necessary for the importer

Here is where we start getting in the meat of things! A TomboloDigitalConnector importer is a java class that inherits all the methods from the AbstractImporter class which do all the necessary housekeeping needed for data crunching. 

* We need to specify this inheritance structure of our importer we do this 
     by extending the ObeseImporter class. IntelliJ can save us the trouble of 
 writing everything manually


In [8]:
HTML('<img src="gifsImporterTut/output_YH0qSo.gif" height="500" width="500">')

You will notice (from the red underlying) that we are not quite there yet. For our class to inherit all methods from the AbstractImporter class we need to implemend a config method that matches the same method of the AbstractImporter class. To check out what methods are implemented in the AbstractImporter class you can always cmd + b on the line that you want to know more about. This will navigate you to the relevant code within DC.

In [9]:
HTML('<img src="gifsImporterTut/output_Leel21.gif" height="500" width="500">')


Here is what the code looks like in the end:

```java
public class ObeseImporter extends AbstractImporter {
    public ObeseImporter(Config config) {
        super(config);
    }

    @Override
    public Provider getProvider() {
        return null;
    }

    @Override
    public DatasourceSpec getDatasourceSpec(String datasourceId) throws Exception {
        return null;
    }

    @Override
    protected void importDatasource(Datasource datasource, List<String> geographyScope, List<String> temporalScope, List<String> datasourceLocation) throws Exception {

    }
}
``` 

Lets try to break things down:
*     
```java
public Provider getProvider()
```
You can think of this method as the metadata for our data provider. It takes two arguments, a label and a name. These can be whatever you want, but its *very* good practice to make it as informative as possible. Usually the label is the general url of the dataset source and the label is the datasource's provider name. In our case, we got the website from activepeople.sportengland.org and the name will be Sports England. We implement that by making a Provider object with those attributes, which we then pass to the getProvider() method:

```java
    protected static final Provider PROVIDER = new Provider(
            "activepeople.sportengland.org",
            "Sports England"
    );
    
    @Override
    public Provider getProvider() {
        return PROVIDER;
    }    
```

 
```java
public DatasourceSpec getDatasourceSpec(String datasourceId)
```

This method is specifying our datasource specifications. It takes as arguments a datasourceId as string, which will be the datasourceId you will be calling the specific dataset of the importer through the recipe file. We do this by implementing a DatasourceSpec with arguments the importer class, the name of the datasource, an ID, a description, and the dataset url:



```java
    @Override
    public DatasourceSpec getDatasourceSpec(String datasourceId) throws Exception {
        DatasourceSpec datasourceSpec = new DatasourceSpec(
                ObeseImporter.class,
                "BMIObese",
                "BMI Obese",
                "% of people with BMI 30-39.9 kg/m^2",
                DATASOURCE);
        return datasourceSpec;
    }
```
Note the <span style="color:purple">***DATASOURCE***</span>  variable. This is a string contains the link to our dataset (.csv file). This could also be the path to the local file if there was one. We need to include it in our importer:
```java
    private static final String DATASOURCE = "http://activepeople.sportengland.org/Result/ExportTable?Id=134820&TabDimension=2&RowDimension=1&ColDimension=4&SelectedTabs[0]=10&ValueMode=0";
```
Finally we need to add the datasourceId in our config constructor. This way we can call this through our recipe language:

```java
    public ObeseImporter(Config config) {
        super(config);
        try {
            datasourceIds = Arrays.asList(getDatasourceSpec("BMIObese").getId());
        } catch (Exception e) {
            e.printStackTrace();
        }
    }
```
There is a final piece of the puzzle that we haven't addressed yet: 
```java
    @Override
    protected void importDatasource(Datasource datasource, List<String> geographyScope, List<String> temporalScope, List<String> datasourceLocation) throws Exception {

    }
```


This method is the meat of the importer so we will dedicate a special step in the tutorial for that.

## Step 4: Implement the importDatasource method

So far were mainly doing housekeeping. Now we will implement the code that reads our dataset file, assigns geometry to subjects, defines te attributes and imports the values. Note that we need to have an understanding on how our data are represented in the dataset and in what format are represented (eg. text, numbers) etc.

The best way to do this is by looking at the dataset:

In [16]:
HTML('<img src="gifsImporterTut/Screen Shot 2017-12-13 at 14.47.34.png" height="1000" width="1000">')


You can see that the dataset is very unstructured. There are superflous lines, values encoded as strings where there shouldn't, local authorities having no spatial extent.

Let's take things one at a time:

* First we need to specify placeholder objects for our SubjectType which will hold the geometry of our observations. As you noticed, we only have the local authority name without any geometry information. We can get that information by querring DC's OaImporter. Before we get to do that we need to instantiate it:

```java
    @Override
    protected void importDatasource(Datasource datasource, List<String> geographyScope, List<String> temporalScope, List<String> datasourceLocation) throws Exception {
       SubjectType localauthority = SubjectTypeUtils.getOrCreate(AbstractONSImporter.PROVIDER,
                        OaImporter.OaType.localAuthority.name(),   OaImporter.OaType.localAuthority.datasourceSpec.getDescription());

    }
```
* Then we need to download the dataset and pass it to an object. We do that by instantiating the format, grab the URL from the getDatasourceSpec("BMIObese") method we defined and pass it into an InputStreamReader object. Here we are using the DownloadUtils class of DC that lets us access the fetchInputStream method that takes care the downloading bit. Once the importer is ready, you can check whether the .csv file was downloaded succesfully by looking your in your /tmp/TomboloData folder. If everything went OK, DC should have assigned a unique ID (SHA-key) for that file: 

```java
//// more code...

    @Override
    protected void importDatasource(Datasource datasource, List<String> geographyScope, List<String> temporalScope, List<String> datasourceLocation) throws Exception {
    
        // We create SubjectType object that we will use to get the appropriate geometries
        // from OaImporter class
       SubjectType localauthority = SubjectTypeUtils.getOrCreate(AbstractONSImporter.PROVIDER,
                        OaImporter.OaType.localAuthority.name(),   OaImporter.OaType.localAuthority.datasourceSpec.getDescription());
                        
                        
        // The code below fetches the .csv file from the URL we specified in our DatasourceSpec object
        CSVFormat format = CSVFormat.DEFAULT;
        String fileLocation = getDatasourceSpec("BMIObese").getUrl();

        URL url;
        try {
            url = new URL(fileLocation);
        } catch (MalformedURLException e) {
            File file;
            if (!(file = new File(fileLocation)).exists()) {
                System.out.println("ERROR: File does not exist: " + fileLocation);
            }
            url = file.toURI().toURL();
        }

        InputStreamReader isr = new InputStreamReader(
                downloadUtils.fetchInputStream(url, getProvider().getLabel(), ".csv"));
                
        // Parsing our csv file
        CSVParser csvFileParser = new CSVParser(isr, format);
        csvRecords = csvFileParser.getRecords();

    }
    
//// more code...

```

Before we finish with this step, we need to instantiate a list for our csvRecords. We do that on the top level of our importer:

```java
public class ObeseImporter extends AbstractImporter {

    // Instantiating the list that will hold our .csv rows
    private List csvRecords;
//// more code...
```

* Now its time to import the dataset in the Postgres database. First, we need to create a list that will hold our vaules. To keep things simple, we will use the FixedValue type for this particular dataset:


```java
//// more code...
    @Override
    protected void importDatasource(Datasource datasource, List<String> geographyScope, List<String> temporalScope, List<String> datasourceLocation) throws Exception {
    
        // We create SubjectType object that we will use to get the appropriate geometries
        // from OaImporter class
        SubjectType localauthority = SubjectTypeUtils.getOrCreate(AbstractONSImporter.PROVIDER,
                OaImporter.OaType.localAuthority.name(), OaImporter.OaType.localAuthority.datasourceSpec.getDescription());

        // We create an empty list that will keep our .csv values
        List<FixedValue> fixedValues = new ArrayList<FixedValue>();
        
//// more code...
```

* Next, we define our Attributes (column names). We do this by implementing a getFixedValueAttributes method in our importer. This will simply assign a name for each of the columns of interest in our dataset. In this case, we have essentially 3 columns of interest: The BMI obesity percentages for different time periods. We exclude the local authorities name as we will fetch that using DC's OAImporter:

```java
//// more code...
    @Override
    public List<Attribute> getFixedValueAttributes(String datasourceID) {
        // Creating a placeholder for our attributes
        List<Attribute> attributes = new ArrayList<>();
        
        // Dataset specific: we hardcode the columns names for the our .csv file
        String[] elements = { "BMI_obesity_2013", "BMI_obesity_2014", "BMI_obesity_2015"};

        // We loop through the elements of the elements object and adding an Attribute object in the list 
        // with nour column names.
        for( int i = 0; i < elements.length; i++) {
            attributes.add(new Attribute(getProvider(), elements[i], elements[i]));

        }
        return attributes;
    }

//// more code...

```

Note that instead of hardcoding the column names we could have read them from our .csv file instead. 


* Then, we create the loop that will read through the .csv lines and pass them in the appropriate placeholders (Attribute for the name of the column, and fixedValues for the actual values of the .csv). You will see lots of try/catch clauses to account for the inconsistencies in the dataset:  
```java
//// more code...
    // We discard the first 6 records in our data file as these don't hold any meaningfull information.
        // We do this  calling an iterator object and simply ignoring them:
        Iterator<CSVRecord> rowIterator = csvRecords.iterator();

        // skipping first 6 rows
        rowIterator.next();
        rowIterator.next();
        rowIterator.next();
        rowIterator.next();
        rowIterator.next();
        rowIterator.next();

        // Looping through the rows of the .csv file
        while (rowIterator.hasNext()){
            CSVRecord row = rowIterator.next();

            // Fetching the subject geometry from OaImporter to save it in getFixedValueAttributes. Note that this
            // corresponds to the 3rd element of our row: row.get(2).
            try{
                Subject subject = SubjectUtils.getSubjectByTypeAndName(localauthority, String.valueOf(row.get(2)).trim());

                // Checking not matched geometries
                if (subject!=null){
                    // Dataset specific: attributeIndex is the column index that we are interested in.
                    int attributeIndex = 3;

                    // The value is a string in our .csv file. We need to clean it and convert it to a number
                    // We need to check for invalid rows so we will suround this with a try catch clause
                    try {
                        String record = row.get(attributeIndex).replace("%","");

                        // We discard the rows that contain no values. In the .csv these are depicted as '*'
                        if (!Objects.equals(record, "*")){
                            System.out.println(record);
                            
                            // The value is a string in our .csv file. We need to clean it and convert it to a 
                            // number. We need to check for invalid rows so we will suround this 
                            // with a try catch clause
                            for (Attribute attribute : datasource.getFixedValueAttributes()) {
                                fixedValues.add(new FixedValue(
                                        subject,
                                        attribute,
                                        record));

                                // We increment to get the rest of the values in the row
                                attributeIndex++;

                            }
                        }
                    } catch (ArrayIndexOutOfBoundsException npe) {
                        System.out.println("INFO - Found invalid row: Skipping");

                    }
                }

                else {
                    System.out.println("INFO - Geometry not found for "+ row.get(2) + ": Skipping");
                    continue;
                }
            }
            // We need to check for again for invalid rows when looping through the local authorities names
            catch (ArrayIndexOutOfBoundsException npe){
                System.out.println("INFO - Found invalid local authority row: Skipping");
                continue;
            }

        }

        // Finally we save the values in the database
        FixedValueUtils.save(fixedValues);
        fixedValues.clear();
//// more code...
```


## **And that's it we are finished! Below is the full importer class**
```java
package uk.org.tombolo.importer.sportsen;


/**
 * Created by tbantis on 13/12/2017.
 */
public class ObeseImporter extends AbstractImporter {

    // Instantiating the list that will hold our .csv rows
    private List csvRecords;
    
    // Instantiating the link to our .csv file
    private static final String DATASOURCE = "http://activepeople.sportengland.org/Result/ExportTable?Id=134820&TabDimension=2&RowDimension=1&ColDimension=4&SelectedTabs[0]=10&ValueMode=0";

    // Instantiating the AbstractImporter constructor
    public ObeseImporter(Config config) {
        super(config);
        try {
            // Specifying the datasourceId. This will be used by the DC recipe
            datasourceIds = Arrays.asList(getDatasourceSpec("BMIObese").getId());
        } catch (Exception e) {
            e.printStackTrace();
        }
    }

    // Instantiating the data Provider
    protected static final Provider PROVIDER = new Provider(
            "activepeople.sportengland.org",
            "Sports England"
    );

    // Getting the data Provider
    @Override
    public Provider getProvider() {
        return PROVIDER;
    }

    // Instantiating the datasoure specifications. 
    @Override
    public DatasourceSpec getDatasourceSpec(String datasourceId) throws Exception {
        DatasourceSpec datasourceSpec = new DatasourceSpec(
                ObeseImporter.class,
                "BMIObese",
                "BMI Obese",
                "% of people with BMI 30-39.9 kg/m^2",
                DATASOURCE);
        return datasourceSpec;
    }

    // The method that does the data  fetching, cleaning, reformatting and importing
    @Override
    protected void importDatasource(Datasource datasource, List<String> geographyScope, List<String> temporalScope, List<String> datasourceLocation) throws Exception {

        // We create SubjectType object that we will use to get the appropriate geometries
        // from OaImporter class
        SubjectType localauthority = SubjectTypeUtils.getOrCreate(AbstractONSImporter.PROVIDER,
                OaImporter.OaType.localAuthority.name(), OaImporter.OaType.localAuthority.datasourceSpec.getDescription());

        // We create an empty list that will keep our .csv values
        List<FixedValue> fixedValues = new ArrayList<FixedValue>();

        CSVFormat format = CSVFormat.DEFAULT;
        String fileLocation = getDatasourceSpec("BMIObese").getUrl();

        // The code below fetches the .csv file from the URL we specified in our DatasourceSpec object
        URL url;
        try {
            url = new URL(fileLocation);
        } catch (MalformedURLException e) {
            File file;
            if (!(file = new File(fileLocation)).exists()) {
                System.out.println("ERROR: File does not exist: " + fileLocation);
            }
            url = file.toURI().toURL();
        }

        InputStreamReader isr = new InputStreamReader(
                downloadUtils.fetchInputStream(url, getProvider().getLabel(), ".csv"));

        // Parsing our csv file
        CSVParser csvFileParser = new CSVParser(isr, format);
        csvRecords = csvFileParser.getRecords();


        // We discard the first 6 records in our data file as these don't hold any meaningfull information.
        // We do this  calling an iterator object and simply ignoring them:
        Iterator<CSVRecord> rowIterator = csvRecords.iterator();

        // skipping first 6 rows
        rowIterator.next();
        rowIterator.next();
        rowIterator.next();
        rowIterator.next();
        rowIterator.next();
        rowIterator.next();

        // Looping through the rows of the .csv file
        while (rowIterator.hasNext()){
            CSVRecord row = rowIterator.next();

            // Fetching the subject geometry from OaImporter to save it in getFixedValueAttributes. Note that this
            // corresponds to the 3rd element of our row: row.get(2).
            try{
                Subject subject = SubjectUtils.getSubjectByTypeAndName(localauthority, String.valueOf(row.get(2)).trim());

                // Checking not matched geometries
                if (subject!=null){
                    // Dataset specific: attributeIndex is the column index that we are interested in.
                    int attributeIndex = 3;

                    // The value is a string in our .csv file. We need to clean it before using it.
                    // We  need to check for invalid rows so we will suround this with a try catch clause
                    try {
                        String record = row.get(attributeIndex).replace("%","");

                        // We discard the rows that contain no values. In the .csv these are depicted as '*'
                        if (!Objects.equals(record, "*")){
                            System.out.println(record);
                            // Here is where we are assigning the values of our .csv file to the attribute fields we
                            // created.
                            for (Attribute attribute : datasource.getFixedValueAttributes()) {
                                fixedValues.add(new FixedValue(
                                        subject,
                                        attribute,
                                        record));

                                // We increment to get the rest of the values in the row
                                attributeIndex++;

                            }
                        }
                    // Catching invalid rows
                    } catch (ArrayIndexOutOfBoundsException npe) {
                        System.out.println("INFO - Found invalid row: Skipping");

                    }
                }
                // Catching invalid geometries
                else {
                    System.out.println("INFO - Geometry not found for "+ row.get(2) + ": Skipping");
                    continue;
                }
            }
            // We need to check for again for invalid rows when looping through the local authorities names
            catch (ArrayIndexOutOfBoundsException npe){
                System.out.println("INFO - Found invalid local authority row: Skipping");
                continue;
            }

        }

        // Finally we save the values in the database
        FixedValueUtils.save(fixedValues);
        fixedValues.clear();

    }

    @Override
    public List<Attribute> getFixedValueAttributes(String datasourceID) {
        // Creating a placeholder for our attributes
        List<Attribute> attributes = new ArrayList<>();

        // Dataset specific: we hardcode the columns names for the our .csv file
        String[] elements = { "BMI_obesity_2013", "BMI_obesity_2014", "BMI_obesity_2015"};

        // We loop through the elements of the elements object and adding an Attribute object in the list
        // with nour column names.
        for( int i = 0; i < elements.length; i++) {
            attributes.add(new Attribute(getProvider(), elements[i], elements[i]));

        }
        return attributes;
    }

}


```


Here is how you could use this importer in a recipe:
```java
  {
  "dataset": {
    "subjects": [
      {
        "subjectType": "localAuthority",
        "provider": "uk.gov.ons"
        ,
        "matchRule": {
          "attribute": "label",
          "pattern": "E090000%"
        }
      }
    ],
    "datasources": [
      {
        "importerClass": "uk.org.tombolo.importer.ons.OaImporter",
        "datasourceId": "localAuthority"
      },
      {
        "importerClass": "uk.org.tombolo.importer.sportsen.ObeseImporter",
        "datasourceId": "BMIObese"

      }

    ],
    "fields": [
      {

          "fieldClass": "uk.org.tombolo.field.value.FixedValueField",
          "label": "BMI_obesity_2013",
          "attribute": {
            "provider": "activepeople.sportengland.org",
            "label": "BMI_obesity_2013"
        }
     }
    ]
  },
    "exporter": "uk.org.tombolo.exporter.GeoJsonExporter"
}
    
```

